# Importing Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# Loading Data, Creating Target variable, and Preprocessing

## Loading data

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d akhilrajcv/bondora-raw

bondora-raw.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/bondora-raw.zip')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
df = pd.read_csv('/content/Bondora_raw.csv')
df.sample(2)

,ReportAsOfEOD,LoanId,LoanNumber,ListedOnUTC,BiddingStartedOn,BidsPortfolioManager,BidsApi,BidsManual,UserName,NewCreditCustomer,LoanApplicationStartedDate,LoanDate,ContractEndDate,FirstPaymentDate,MaturityDate_Original,MaturityDate_Last,ApplicationSignedHour,ApplicationSignedWeekday,VerificationType,LanguageCode,Age,DateOfBirth,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,County,City,UseOfLoan,Education,MaritalStatus,NrOfDependants,EmploymentStatus,EmploymentDurationCurrentEmployer,EmploymentPosition,WorkExperience,OccupationArea,HomeOwnershipType,IncomeFromPrincipalEmployer,IncomeFromPension,IncomeFromFamilyAllowance,IncomeFromSocialWelfare,IncomeFromLeavePay,IncomeFromChildSupport,IncomeOther,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,RefinanceLiabilities,DebtToIncome,FreeCash,MonthlyPaymentDay,ActiveScheduleFirstPaymentReached,PlannedPrincipalTillDate,PlannedInterestTillDate,LastPaymentOn,CurrentDebtDaysPrimary,DebtOccuredOn,CurrentDebtDaysSecondary,DebtOccuredOnForSecondary,ExpectedLoss,LossGivenDefault,ExpectedReturn,ProbabilityOfDefault,DefaultDate,PrincipalOverdueBySchedule,PlannedPrincipalPostDefault,PlannedInterestPostDefault,EAD1,EAD2,PrincipalRecovery,InterestRecovery,RecoveryStage,StageActiveSince,ModelVersion,Rating,EL_V0,Rating_V0,EL_V1,Rating_V1,Rating_V2,Status,Restructured,ActiveLateCategory,WorseLateCategory,CreditScoreEsMicroL,CreditScoreEsEquifaxRisk,CreditScoreFiAsiakasTietoRiskGrade,CreditScoreEeMini,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,GracePeriodStart,GracePeriodEnd,NextPaymentDate,NextPaymentNr,NrOfScheduledPayments,ReScheduledOn,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory
4987,2020-01-27,17A46583-E4BC-4A9B-98D2-A47B01191D9B,358583,2015-04-15 17:07:02,2015-04-15 17:07:02,510,0,490.0,BO33742K3,True,2015-04-15 17:03:30,2015-04-20,NaN,2015-06-08,2020-05-08,2020-05-08,17,4,3.0,6,35,1980-03-25,0.0,ES,1000.0,1000.0,57.09,60,54.03,BARCELONA,BARCELONA,2,5.0,1.0,0,3.0,MoreThan5Years,MiddleManager,10To15Years,10.0,4.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,4000.0,3,1709.0,0,14.08,2236.97,8,True,284.05,819.55,2020-01-13,1539.0,2015-11-09,1601.0,2015-09-08,0.365178,0.90,0.205701,0.312118,2015-12-15,608.8,794.07,1689.19,980.58,767.85,191.33,0.0,2.0,2019-04-24 00:00:00,1.0,HR,NaN,NaN,0.365178,HR,HR,Late,False,180+,180+,M1,C,NaN,NaN,210.75,212.73,0.0,0.0,789.25,4642.2,0,0.0,0.00,0.0,0,NaN,NaN,NaN,0.0,60.0,NaN,0.0,0.91,8-15
19210,2020-01-27,CF0F19F5-F1DA-4C35-9D9E-A6D4014C1521,579520,2016-12-07 05:59:59,2016-12-07 07:59:59,1680,1500,5.0,BOK11K233,False,2016-12-06 20:09:04,2016-12-07,2019-12-09,2017-01-09,2019-12-09,2019-12-09,20,3,4.0,1,54,1962-11-10,1.0,EE,3185.0,3185.0,26.93,36,140.53,IDA-VIRUMAA,KOHTLA-JÄRVE,6,4.0,1.0,NaN,3.0,UpTo3Years,MiddleManager,MoreThan25Years,7.0,1.0,600.0,191.0,0.0,0.0,0.0,0.0,0.0,791.0,4,601.0,0,62.14,49.47,9,True,1192.89,3185.00,2019-12-09,NaN,NaN,NaN,NaN,0.095687,0.58,0.132212,0.180653,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,D,NaN,NaN,NaN,NaN,NaN,Repaid,False,NaN,NaN,NaN,NaN,NaN,900.0,3185.00,1496.45,0.0,0.0,0.00,0.0,1,500.0,501.62,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.00,NaN


In [ ]:
df = df[df['Status'] != 'Current']
df.shape

(77394, 112)

# Pipeline

## The Setup

### Data preprocessing

In [ ]:
# Transform [DefaultDate] --> [LoanStatus]
df['DefaultDate'] = np.where(df['DefaultDate'].isnull(), 'NonDefault', 'Default')
df.rename(columns={'DefaultDate': 'LoanStatus'}, inplace=True)

# Filtering according to age(age >= 18)
df = df[df['Age'] >= 18]

# Drop columns with more than 40% null values
cols_to_drop = df.columns[df.isnull().mean() > 0.4]
df.drop(columns=cols_to_drop, axis=1, inplace=True)

# Drop irrelavent columns
df.drop(columns=['ReportAsOfEOD', 'LoanId', 'LoanNumber', 'ListedOnUTC', 'DateOfBirth', 'BiddingStartedOn', 'UserName', 'IncomeFromPrincipalEmployer',
                 'IncomeFromPension', 'IncomeFromFamilyAllowance', 'IncomeFromSocialWelfare', 'IncomeFromLeavePay', 'IncomeFromChildSupport',
                 'IncomeOther', 'LoanApplicationStartedDate', 'ApplicationSignedHour', 'ApplicationSignedWeekday', 'WorseLateCategory',
                 'ActiveScheduleFirstPaymentReached', 'PlannedInterestTillDate', 'LastPaymentOn', 'ExpectedLoss', 'LossGivenDefault',
                 'ExpectedReturn', 'ProbabilityOfDefault', 'PrincipalOverdueBySchedule', 'StageActiveSince', 'ModelVersion', 'LoanDate',
                 'FirstPaymentDate', 'MaturityDate_Original',	'MaturityDate_Last', 'County',	'City', 'DebtOccuredOn',
                 'DebtOccuredOnForSecondary'], axis=1, inplace=True)

df.shape

(77341, 52)

In [ ]:
df.sample(2)

,BidsPortfolioManager,BidsApi,BidsManual,NewCreditCustomer,VerificationType,LanguageCode,Age,Gender,Country,AppliedAmount,Amount,Interest,LoanDuration,MonthlyPayment,UseOfLoan,Education,MaritalStatus,EmploymentStatus,EmploymentDurationCurrentEmployer,OccupationArea,HomeOwnershipType,IncomeTotal,ExistingLiabilities,LiabilitiesTotal,RefinanceLiabilities,DebtToIncome,FreeCash,MonthlyPaymentDay,PlannedPrincipalTillDate,CurrentDebtDaysPrimary,CurrentDebtDaysSecondary,LoanStatus,RecoveryStage,Rating,Status,Restructured,ActiveLateCategory,CreditScoreEsMicroL,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,NoOfPreviousLoansBeforeLoan,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PreviousEarlyRepaymentsCountBeforeLoan,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,ActiveLateLastPaymentCategory
1369,735,0,165.0,True,3.0,4,24,1.0,FI,900.0,900.0,21.76,36,37.26,0,4.0,3.0,3.0,UpTo5Years,9.0,3.0,1551.0,4,870.0,0,26.26,643.74,27,566.16,NaN,NaN,Default,NaN,C,Repaid,True,NaN,NaN,900.0,479.21,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,NaN
66043,369,0,1226.0,True,4.0,3,40,0.0,EE,2125.0,2125.0,29.94,36,97.22,-1,3.0,-1.0,-1.0,UpTo5Years,-1.0,2.0,1500.0,2,508.0,0,0.00,0.00,1,NaN,NaN,NaN,NonDefault,NaN,D,Repaid,False,NaN,M,2125.0,253.95,NaN,NaN,0.0,0.0,0,0.0,NaN,NaN,0,NaN,NaN,NaN


In [ ]:
df['RecoveryStage'].isnull().sum()

26838

In [ ]:
df.loc[(df['Status'] == 'Late') & (df['LoanStatus'] == 'Default')].shape

(37111, 52)

### Null values imputation

In [ ]:
# Import required library
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Seperating numerical, categorical features and discrete features
cat_features = cat_features = df.columns[(df.dtypes == 'object') | (df.dtypes == 'bool')].to_list()
discrete_features = df.columns[((df.dtypes == 'int64') | (df.dtypes == 'float64')) & (df.nunique() <= 65)].to_list()
num_features = df.columns[~df.columns.isin(cat_features + discrete_features)].to_list()

cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='median')
dis_imputer = SimpleImputer(strategy='most_frequent')

# Apply imputers separately to categorical and numerical features
imputed_cat = cat_imputer.fit_transform(df.iloc[:, cat_pos])
imputed_num = num_imputer.fit_transform(df.iloc[:, num_pos])
imputed_dis = dis_imputer.fit_transform(df.iloc[:, dis_pos])


# Move 'Age' column from categorical to numerical list
discrete_features.remove('Age')
num_features.append('Age')

# Finding the index of numerical and categorical features
num_pos = [df.columns.get_loc(name) for name in num_features]
cat_pos = [df.columns.get_loc(name) for name in cat_features]
dis_pos = [df.columns.get_loc(name) for name in discrete_features]

cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='median')
dis_imputer = SimpleImputer(strategy='most_frequent')

# Apply imputers separately to categorical and numerical features
imputed_cat = cat_imputer.fit_transform(df.iloc[:, cat_pos])
imputed_num = num_imputer.fit_transform(df.iloc[:, num_pos])
imputed_dis = dis_imputer.fit_transform(df.iloc[:, dis_pos])

# Create DataFrames for imputed categorical and numerical features
df_imputed_cat = pd.DataFrame(imputed_cat, columns=cat_features)
df_imputed_num = pd.DataFrame(imputed_num, columns=num_features)
df_imputed_dis = pd.DataFrame(imputed_dis, columns=discrete_features)

# Concatenate the imputed DataFrames back together
df = pd.concat([df_imputed_cat, df_imputed_num, df_imputed_dis], axis=1)

# Print the data types of columns in the original and imputed DataFrames
df.sample(2)

,NewCreditCustomer,Country,EmploymentDurationCurrentEmployer,LoanStatus,Rating,Status,Restructured,ActiveLateCategory,CreditScoreEsMicroL,ActiveLateLastPaymentCategory,BidsPortfolioManager,BidsApi,BidsManual,AppliedAmount,Amount,Interest,MonthlyPayment,IncomeTotal,LiabilitiesTotal,DebtToIncome,FreeCash,PlannedPrincipalTillDate,CurrentDebtDaysPrimary,CurrentDebtDaysSecondary,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,Age,VerificationType,LanguageCode,Gender,LoanDuration,UseOfLoan,Education,MaritalStatus,EmploymentStatus,OccupationArea,HomeOwnershipType,ExistingLiabilities,RefinanceLiabilities,MonthlyPaymentDay,RecoveryStage,NoOfPreviousLoansBeforeLoan,PreviousEarlyRepaymentsCountBeforeLoan
44042,True,FI,MoreThan5Years,Default,F,Repaid,False,180+,M,180+,1999.0,0.0,695.0,6485.0,6485.0,51.50,342.64,5500.0,1100.0,0.0,0.00,530.00,415.0,458.0,6485.00,1298.79,0.0,0.0,0.00,0.00,0.0,41.36,0.0,0.0,0.00,43.0,1.0,4.0,1.0,48.0,-1.0,5.0,-1.0,-1.0,-1.0,8.0,1.0,0.0,13.0,2.0,0.0,0.0
9514,True,FI,UpTo1Year,Default,HR,Late,False,180+,M,61-90,2175.0,0.0,55.0,5845.0,2230.0,40.59,241.47,1826.0,1721.0,69.3,13.53,539.69,996.0,996.0,780.39,1110.19,0.0,0.0,1449.61,1752.11,0.0,0.00,0.0,0.0,298.18,24.0,4.0,4.0,0.0,60.0,7.0,3.0,3.0,3.0,8.0,4.0,12.0,1.0,5.0,2.0,0.0,0.0


### Outlier Treatment

In [ ]:
cols = df.columns[((df.dtypes == 'float64') | (df.dtypes == 'int64')) & (df.nunique() > 65)]
cols

Index(['BidsPortfolioManager', 'BidsApi', 'BidsManual', 'AppliedAmount',
       'Amount', 'Interest', 'MonthlyPayment', 'IncomeTotal',
       'LiabilitiesTotal', 'DebtToIncome', 'FreeCash',
       'PlannedPrincipalTillDate', 'CurrentDebtDaysPrimary',
       'CurrentDebtDaysSecondary', 'PrincipalPaymentsMade',
       'InterestAndPenaltyPaymentsMade', 'PrincipalWriteOffs',
       'InterestAndPenaltyWriteOffs', 'PrincipalBalance',
       'InterestAndPenaltyBalance', 'AmountOfPreviousLoansBeforeLoan',
       'PreviousRepaymentsBeforeLoan', 'PreviousEarlyRepaymentsBefoleLoan',
       'PrincipalDebtServicingCost', 'InterestAndPenaltyDebtServicingCost'],
      dtype='object')

In [ ]:
# Outliers treatment
def cap_outliers(df):
  for ft in cols:
    q1 = df[ft].quantile(0.25)
    q3 = df[ft].quantile(0.75)
    iqr = q3 - q1

    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)

    df[ft] = np.where(df[ft] > upper_bound, upper_bound, np.where(df[ft] < lower_bound, lower_bound, df[ft]))

  return df

In [ ]:
df = cap_outliers(df)
df.shape

(77341, 52)

### Feature Creation

In [ ]:
# LoanTenure
df.rename(columns={'LoanDuration':'LoanTenure'}, inplace = True)

# EMI
df['EMI'] = np.round((df['Amount'] * ((df['Interest'] / 12) / 100) * (1 + ((df['Interest'] / 12) / 100)) ** df['LoanTenure']) / (((1 + ((df['Interest'] / 12) / 100)) ** df['LoanTenure']) - 1), 3)

# ELA
df['MaxAllowableAmount'] = ((df['IncomeTotal'] - df['LiabilitiesTotal']) * 0.30)
df['TotalPaymentsDue'] = (df['AppliedAmount'] + (df['AppliedAmount'] * (df['Interest']/ 100))) * df['LoanTenure']
df['TotalPaymentsDue'] = (df['AppliedAmount'] + (df['AppliedAmount'] * ((df['Interest'] / 12 )/ 100) * df['LoanTenure']))
df['ELA'] = np.where(df['TotalPaymentsDue'] <= df['MaxAllowableAmount'], df['AppliedAmount'], df['MaxAllowableAmount'])
df.drop(columns=['MaxAllowableAmount', 'TotalPaymentsDue'], axis=1, inplace=True)

# PROI
df['InterestAmount'] = (df['Amount']*(df['Interest']/100))
df['TotalAmount'] = (df['InterestAmount'] + df['Amount'])
df['ROI'] = (df['InterestAmount'] / df['TotalAmount'])*100

df['PROI'] = df['ROI'].median()

# Check for loan tenure
df['PROI'] = np.where(df['LoanTenure'] <= 19, df['PROI'] - 5,
                        np.where(df['LoanTenure'] > 25, df['PROI'] + 5, df['PROI']))

# Check for DebtToIncome
df['PROI'] = np.where(df['DebtToIncome'] == 0, df['PROI'] - 5, df['PROI'] + 5)

# Check for IncomeTotal
df['PROI'] = np.where(df['IncomeTotal'] <= 1000, df['PROI'] - 5, df['PROI'])

# Check for AppliedAmount
df['PROI'] = np.where((df['AppliedAmount'] >= 850) & (df['AppliedAmount'] < 1175), df['PROI'] - 5,
                        np.where(df['AppliedAmount'] >= 2000, df['PROI'] + 5, df['PROI']))

df.drop(columns=['TotalAmount', 'InterestAmount', 'ROI'], axis=1, inplace=True)
df.sample(2)

,NewCreditCustomer,Country,EmploymentDurationCurrentEmployer,LoanStatus,Rating,Status,Restructured,ActiveLateCategory,CreditScoreEsMicroL,ActiveLateLastPaymentCategory,BidsPortfolioManager,BidsApi,BidsManual,AppliedAmount,Amount,Interest,MonthlyPayment,IncomeTotal,LiabilitiesTotal,DebtToIncome,FreeCash,PlannedPrincipalTillDate,CurrentDebtDaysPrimary,CurrentDebtDaysSecondary,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,Age,VerificationType,LanguageCode,Gender,LoanTenure,UseOfLoan,Education,MaritalStatus,EmploymentStatus,OccupationArea,HomeOwnershipType,ExistingLiabilities,RefinanceLiabilities,MonthlyPaymentDay,RecoveryStage,NoOfPreviousLoansBeforeLoan,PreviousEarlyRepaymentsCountBeforeLoan,EMI,ELA,PROI
57148,False,EE,MoreThan5Years,NonDefault,B,Repaid,False,180+,M,180+,380.0,12.5,1834.0,3615.0,3615.0,13.67,135.02,3665.0,1670.44,0.0,0.00,530.00,415.0,458.0,3532.95,270.99,0.0,0.0,0.00,0.00,7500.0,921.55,0.0,0.0,0.0,41.0,1.0,1.0,0.0,36.0,-1.0,5.0,-1.0,-1.0,-1.0,1.0,3.0,0.0,12.0,1.0,7.0,0.0,122.974,598.368,29.230944
4716,True,EE,MoreThan5Years,Default,D,Late,False,180+,M,180+,1370.0,0.0,630.0,2000.0,2000.0,29.62,70.91,336.0,398.16,21.1,15.09,847.48,1163.0,1301.0,681.37,439.12,0.0,0.0,1318.63,1478.22,0.0,0.00,0.0,0.0,0.0,67.0,1.0,1.0,0.0,60.0,0.0,3.0,4.0,6.0,1.0,1.0,3.0,2.0,5.0,2.0,0.0,4.0,64.241,-18.648,34.230944


In [ ]:
df.shape

(77341, 55)

In [ ]:
df['PROI'].describe()

count    77341.000000
mean        27.686873
std          7.012408
min          4.230944
25%         24.230944
50%         29.230944
75%         29.230944
max         39.230944
Name: PROI, dtype: float64

### Data Encoding

In [ ]:
def label_encode_columns_inplace(df, columns_to_encode):
    for column in columns_to_encode:
        unique_values = df[column].unique()
        encoding = {value: index for index, value in enumerate(unique_values)}
        df[column] = df[column].map(encoding)


# List of columns to label encode
columns_to_encode = df.columns[(df.dtypes == 'object')].to_list()

label_encode_columns_inplace(df, columns_to_encode)

# Display the modified DataFrame
df.head()

,NewCreditCustomer,Country,EmploymentDurationCurrentEmployer,LoanStatus,Rating,Status,Restructured,ActiveLateCategory,CreditScoreEsMicroL,ActiveLateLastPaymentCategory,BidsPortfolioManager,BidsApi,BidsManual,AppliedAmount,Amount,Interest,MonthlyPayment,IncomeTotal,LiabilitiesTotal,DebtToIncome,FreeCash,PlannedPrincipalTillDate,CurrentDebtDaysPrimary,CurrentDebtDaysSecondary,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,Age,VerificationType,LanguageCode,Gender,LoanTenure,UseOfLoan,Education,MaritalStatus,EmploymentStatus,OccupationArea,HomeOwnershipType,ExistingLiabilities,RefinanceLiabilities,MonthlyPaymentDay,RecoveryStage,NoOfPreviousLoansBeforeLoan,PreviousEarlyRepaymentsCountBeforeLoan,EMI,ELA,PROI
0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,115.0410,319.5582,115.0408,30.0,101.15,3665.0,0.0,0.0,0.0,1221.4300,415.0,458.0,115.0408,20.4222,0.0,0.0,0.00,0.00,83.0852,0.0000,0.0,0.0,0.0,61.0,2.0,1.0,1.0,12.0,7.0,3.0,1.0,3.0,7.0,1.0,0.0,0.0,25.0,2.0,1.0,0.0,11.215,319.5582,14.230944
1,1,0,1,0,0,0,0,0,0,0,0.0,0.0,140.6057,191.7349,140.6057,25.0,101.15,3665.0,0.0,0.0,0.0,1221.4300,415.0,458.0,140.6057,2.0227,0.0,0.0,0.00,0.00,255.6467,258.6256,0.0,0.0,0.0,48.0,2.0,1.0,1.0,1.0,7.0,5.0,4.0,3.0,16.0,1.0,0.0,0.0,15.0,2.0,2.0,0.0,143.535,191.7349,14.230944
2,0,0,2,1,0,1,1,0,0,0,0.0,0.0,319.5580,319.5582,319.5409,25.0,101.15,3665.0,0.0,0.0,0.0,319.5409,1163.0,1301.0,203.1909,59.7626,0.0,0.0,116.35,414.07,0.0000,0.0000,0.0,0.0,0.0,58.0,2.0,1.0,1.0,20.0,2.0,4.0,1.0,3.0,9.0,1.0,0.0,0.0,25.0,2.0,0.0,0.0,19.700,319.5582,19.230944
3,0,0,3,0,0,0,0,0,0,0,0.0,0.0,57.5205,127.8233,57.5205,45.0,101.15,3665.0,0.0,0.0,0.0,900.0000,415.0,458.0,57.5205,18.7323,0.0,0.0,0.00,0.00,134.2144,0.0000,0.0,0.0,0.0,23.0,2.0,1.0,1.0,15.0,0.0,2.0,3.0,-1.0,1.0,1.0,0.0,0.0,15.0,2.0,1.0,0.0,5.083,127.8233,14.230944
4,0,0,3,1,0,0,0,0,0,0,0.0,0.0,319.5582,319.5582,319.5436,30.0,101.15,3665.0,0.0,0.0,0.0,1221.4300,415.0,1301.0,319.5436,220.4200,0.0,0.0,0.00,0.00,146.9966,0.0000,0.0,0.0,0.0,25.0,2.0,1.0,1.0,12.0,6.0,4.0,2.0,3.0,7.0,1.0,0.0,0.0,25.0,2.0,1.0,0.0,31.151,319.5582,14.230944


LoanStatus  
- 0 -> NonDefault
- 1 -> Default

### Feature Removal

In [ ]:
# Droping featurs with high correlation
corr_matrix = df.corr()

# Set a correlation threshold
correlation_threshold = 0.8

# Find features with correlation above the threshold
high_correlation_features = np.where(np.abs(corr_matrix) > correlation_threshold)

# Get unique feature pairs with high correlation
high_correlation_pairs = [(df.columns[i], df.columns[j]) for i, j in zip(*high_correlation_features) if i != j and i < j]

print("Features with high correlation:")
for feature_pair in high_correlation_pairs:
    print(feature_pair)
    df.drop(columns=feature_pair[0], axis=1, inplace=True)

df.shape

Features with high correlation:
('Country', 'LanguageCode')
('AppliedAmount', 'Amount')
('CurrentDebtDaysPrimary', 'CurrentDebtDaysSecondary')
('MaritalStatus', 'EmploymentStatus')


(77341, 51)

In [ ]:
irrelevent_features = df.columns[df.nunique() == 1]
irrelevent_features

Index(['PrincipalWriteOffs', 'InterestAndPenaltyWriteOffs',
       'PreviousEarlyRepaymentsBefoleLoan', 'PrincipalDebtServicingCost',
       'InterestAndPenaltyDebtServicingCost'],
      dtype='object')

In [ ]:
# After the outlier treatment and null imputation some of the features are left with only 1 category,
# We will remove those too
df.drop(columns=irrelevent_features, axis=1, inplace=True)
df.shape

(77341, 46)

## Classification Pipeline

### Splitting the dataset

In [ ]:
# Train Test Split
from sklearn.model_selection import train_test_split

X = df.drop(columns=['LoanStatus', 'EMI', 'ELA', 'PROI'], axis=1)
y_class = df['LoanStatus']
y_reg = df[['EMI', 'ELA', 'PROI']]

# Classification data
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X, y_class, test_size=0.2, random_state=42, stratify=y)
# Regression data
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)

In [ ]:
# Train Test Split
from sklearn.model_selection import train_test_split

X = df.drop(columns=['LoanStatus', 'EMI', 'ELA', 'PROI'], axis=1)
y = df['LoanStatus']
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train_clf.shape, X_test_clf.shape, y_train_clf.shape

((61872, 42), (15469, 42), (61872,))

### Class Balancing

In [ ]:
y_train_clf.value_counts()

1    34213
0    27659
Name: LoanStatus, dtype: int64

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

X_train_clf, y_train_clf = smote.fit_resample(X_train_clf, y_train_clf)
y_train_clf.value_counts()

0    34213
1    34213
Name: LoanStatus, dtype: int64

### Scaling Transformer

In [ ]:
# Standard Scaler
from sklearn.preprocessing import StandardScaler

trf1 = ColumnTransformer([
    ('scale', StandardScaler(), slice(0, 47))
])

### Feature Selection Transformer

In [ ]:
# scaler = StandardScaler()
# X_train_std = scaler.fit_transform(X_train_clf)

# pca = PCA(n_components=None)
# X_train_trf = pca.fit_transform(X_train_std)

# print(pca.explained_variance_.shape, pca.components_.shape)
# np.cumsum(pca.explained_variance_ratio_)

In [ ]:
# PCA
from sklearn.decomposition import PCA

# 25 components explain 90% of variance and 31 components explain 95% of variance
trf2 = PCA(n_components=25)

### Model Transformer

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

trf3 = LogisticRegression()

### Create Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

class_pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3)
])

In [ ]:
class_pipe.fit(X_train_clf, y_train_clf)

Pipeline(steps=[('trf1',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  slice(0, 47, None))])),
                ('trf2', PCA(n_components=25)),
                ('trf3', LogisticRegression())])

In [ ]:
y_pred_clf = class_pipe.predict(X_test_clf)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

print("\nAccuracy score:\n", round(accuracy_score(y_test_clf, y_pred_clf)*100,2), '%')
print('*'*40)
print("\nConfusion Matrix:\n", confusion_matrix(y_test_clf, y_pred_clf))
print('*'*40)
print("\nClassification Report:\n", classification_report(y_test_clf, y_pred_clf,
                                        target_names=['Default', 'Non-deafault']))


Accuracy score:
 96.12 %
****************************************

Confusion Matrix:
 [[6785  130]
 [ 470 8084]]
****************************************

Classification Report:
               precision    recall  f1-score   support

     Default       0.94      0.98      0.96      6915
Non-deafault       0.98      0.95      0.96      8554

    accuracy                           0.96     15469
   macro avg       0.96      0.96      0.96     15469
weighted avg       0.96      0.96      0.96     15469



In [ ]:
y_probs_clf = class_pipe.predict_proba(X_test_clf)[:, 1]

roc_auc = roc_auc_score(y_test_clf, y_probs_clf)
print("ROC-AUC Score:", roc_auc)

ROC-AUC Score: 0.9846755865632498


## Regression Pipeline

### Splitting the dataset

In [ ]:
# Train Test Split

X = df.drop(columns=['EMI', 'ELA', 'PROI'], axis=1)
y = df[['EMI', 'ELA', 'PROI']]
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_reg.shape, X_test_reg.shape, y_train_reg.shape

((61872, 43), (15469, 43), (61872, 3))

In [ ]:
X_train_reg.columns.get_loc('LoanStatus')

2

### Polynomial Transformation

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Converting features to polynomial features is a technique used to capture
# nonlinear relationships between the features and the target variable.
poly = PolynomialFeatures(degree=2)
X_train_reg = poly.fit_transform(X_train_reg)
X_test_reg = poly.fit_transform(X_test_reg)

### Scaling Transformer

In [ ]:
# Standard Scaler

trf1 = ColumnTransformer([
    ('scale', StandardScaler(), slice(0, 115))
])

### Feature Selection Transformer

In [ ]:
# scaler = StandardScaler()
# X_train_std = scaler.fit_transform(X_train_reg)

# pca = PCA(n_components=None)
# X_train_trf = pca.fit_transform(X_train_std)

# print(pca.explained_variance_.shape, pca.components_.shape)
# np.cumsum(pca.explained_variance_ratio_)

In [ ]:
# PCA

# 26 components explain 90% of variance and 31 components explain 95% of variance
trf2 = PCA(n_components=115)

### Model Transformer

In [ ]:
# Random Forest
from sklearn.linear_model import LinearRegression

trf3 = LinearRegression()

### Create Pipeline

In [ ]:
reg_pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3)
])

In [ ]:
reg_pipe.fit(X_train_reg, y_train_reg)

Pipeline(steps=[('trf1',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  slice(0, 115, None))])),
                ('trf2', PCA(n_components=115)), ('trf3', LinearRegression())])

In [ ]:
y_pred_reg = reg_pipe.predict(X_test_reg)

In [ ]:
# Score and test results
from sklearn.metrics import r2_score

print('R2_score: ', round(r2_score(y_test_reg, y_pred_reg)*100,2), '%')

R2_score:  82.58 %


# Combining Models

In [ ]:
# predicted_classes = class_pipe.predict(X_test_clf)

# # Add predicted classes as a new feature to the original data
# X_test_reg_augmented = X_test_reg.copy()
# X_test_reg_augmented['LoanStatus'] = predicted_classes

# # Fit and evaluate regression pipeline with augmented data
# reg_pipe.fit(X_train_reg, y_train_reg)
# reg_score = reg_pipe.score(X_test_reg_augmented, y_test_reg)
# print("Regression Pipeline Score with Predicted Classes:", reg_score)

# Saving Pipeline

In [ ]:
import pickle

pickle.dump(class_pipe, open('class_pipeline.pkl', 'wb'))
pickle.dump(reg_pipe, open('reg_pipeline.pkl', 'wb'))

# Combining Pipelines

In [ ]:
def pipeline_out(X):
    output = []
    class_pipe = pickle.load(open("class_pipeline.pkl", 'rb'))
    output.append(class_pipe.predict(X))

    # 0 -> NonDefault,    1 -> Default
    if output[-1] == 0:
      reg_pipe = pickle.load(open("reg_pipeline.pkl", 'rb'))
      X_agumented = X.copy()
      X_agumented = np.insert(X_agumented, 2, 0).reshape(1, -1)
      print(X_agumented)
      output.append(reg_pipe.predict(X_agumented))
    else:
      print('The applicant may default!!!')

    return output

# Testing our pipeline

In [ ]:
test_input = np.array(X_train_clf.iloc[701]).reshape(1, -1)
test_input

array([[1.0000e+00, 2.0000e+00, 7.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 5.1500e+02, 0.0000e+00,
        1.5000e+01, 5.3000e+02, 1.0340e+01, 4.8450e+01, 8.8600e+02,
        6.7236e+02, 3.2600e+01, 1.6519e+02, 5.3000e+02, 4.5800e+02,
        5.3000e+02, 3.0740e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 3.7900e+03, 9.2155e+02, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 3.2000e+01, 4.0000e+00, 1.0000e+00, 0.0000e+00,
        1.2000e+01, 3.0000e+00, 5.0000e+00, 3.0000e+00, 1.5000e+01,
        2.0000e+00, 6.0000e+00, 0.0000e+00, 7.0000e+00, 2.0000e+00,
        2.0000e+00, 0.0000e+00]])

In [ ]:
pipeline_out(test_input)

[[1.0000e+00 2.0000e+00 0.0000e+00 7.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 5.1500e+02 0.0000e+00 1.5000e+01
  5.3000e+02 1.0340e+01 4.8450e+01 8.8600e+02 6.7236e+02 3.2600e+01
  1.6519e+02 5.3000e+02 4.5800e+02 5.3000e+02 3.0740e+01 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 3.7900e+03 9.2155e+02 0.0000e+00
  0.0000e+00 0.0000e+00 3.2000e+01 4.0000e+00 1.0000e+00 0.0000e+00
  1.2000e+01 3.0000e+00 5.0000e+00 3.0000e+00 1.5000e+01 2.0000e+00
  6.0000e+00 0.0000e+00 7.0000e+00 2.0000e+00 2.0000e+00 0.0000e+00]]


[array([0]), array([[42.63515295, 37.91124146, 21.44058461]])]

In [ ]:
X_test_reg.sample()

,NewCreditCustomer,EmploymentDurationCurrentEmployer,LoanStatus,Rating,Status,Restructured,ActiveLateCategory,CreditScoreEsMicroL,ActiveLateLastPaymentCategory,BidsPortfolioManager,BidsApi,BidsManual,Amount,Interest,MonthlyPayment,IncomeTotal,LiabilitiesTotal,DebtToIncome,FreeCash,PlannedPrincipalTillDate,CurrentDebtDaysSecondary,PrincipalPaymentsMade,InterestAndPenaltyPaymentsMade,PrincipalWriteOffs,InterestAndPenaltyWriteOffs,PrincipalBalance,InterestAndPenaltyBalance,AmountOfPreviousLoansBeforeLoan,PreviousRepaymentsBeforeLoan,PreviousEarlyRepaymentsBefoleLoan,PrincipalDebtServicingCost,InterestAndPenaltyDebtServicingCost,Age,VerificationType,LanguageCode,Gender,LoanTenure,UseOfLoan,Education,EmploymentStatus,OccupationArea,HomeOwnershipType,ExistingLiabilities,RefinanceLiabilities,MonthlyPaymentDay,RecoveryStage,NoOfPreviousLoansBeforeLoan,PreviousEarlyRepaymentsCountBeforeLoan
29769,0,1,1,7,1,0,0,0,0,228.0,1.0,301.0,530.0,10.85,19.09,982.0,260.0,0.0,0.0,61.69,552.0,113.86,37.17,0.0,0.0,416.14,74.83,0.0,0.0,0.0,0.0,0.0,65.0,4.0,4.0,0.0,36.0,-1.0,2.0,-1.0,-1.0,1.0,1.0,0.0,21.0,2.0,0.0,0.0
